In [1]:
using Pkg

In [2]:
Pkg.add("Flux")

    Updating registry at `C:\Users\amans\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\amans\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\amans\.julia\environments\v1.7\Manifest.toml`


In [3]:
using Flux

In [4]:
x = ones(Float32,1,2,3)

1×2×3 Array{Float32, 3}:
[:, :, 1] =
 1.0  1.0

[:, :, 2] =
 1.0  1.0

[:, :, 3] =
 1.0  1.0

In [5]:
typeof(x)

Array{Float32, 3}

In [6]:
f(x) = 3x^2 + 2x + 1

f (generic function with 1 method)

In [7]:
f(1)

6

In [8]:
df(x) = gradient(f,x)[1]

df (generic function with 1 method)

In [9]:
df(1)

8.0

In [10]:
d2f(x) = gradient(df,x)[1]

d2f (generic function with 1 method)

In [11]:
d2f(1)

6.0

In [12]:
f(x,y) = sum((x.-y).^2)

f (generic function with 2 methods)

In [13]:
gradient(f,[2,1],[2,0])

([0.0, 2.0], [-0.0, -2.0])

In [14]:
x= [2,1];

In [15]:
y = [2,0];

In [16]:
gs = gradient(params(x,y)) do
    f(x,y)
end

Grads(...)

In [17]:
gs[x]

2-element Vector{Float64}:
 0.0
 2.0

In [18]:
gs[y]

2-element Vector{Float64}:
 -0.0
 -2.0

In [19]:
W = rand(2,5)

2×5 Matrix{Float64}:
 0.761199  0.518721  0.299738  0.819246  0.795295
 0.695557  0.294634  0.136187  0.966324  0.556343

In [20]:
b = rand(2)

2-element Vector{Float64}:
 0.14996273490197987
 0.6003021631713606

In [21]:
predict(x) = W*x .+ b

predict (generic function with 1 method)

In [22]:
function loss(x,y)
    y_cap = predict(x)
    sum((y .- y_cap).^2)
end

loss (generic function with 1 method)

In [23]:
x,y = rand(5),rand(2)

([0.7296468340269642, 0.46668665850674407, 0.1263968430225897, 0.8288758645565867, 0.6767460059294579], [0.45303268852343126, 0.5292262928918061])

In [24]:
x

5-element Vector{Float64}:
 0.7296468340269642
 0.46668665850674407
 0.1263968430225897
 0.8288758645565867
 0.6767460059294579

In [25]:
y

2-element Vector{Float64}:
 0.45303268852343126
 0.5292262928918061

In [26]:
loss(x,y)

6.712027970590133

In [27]:
gs = gradient(() -> loss(x,y), params(W,b))

Grads(...)

In [28]:
W̄ = gs[W]

2×5 Matrix{Float64}:
 2.55314  1.633    0.44228  2.90035  2.36803
 2.78837  1.78346  0.48303  3.16758  2.58621

In [29]:
W .-= 0.1 .* W̄

2×5 Matrix{Float64}:
 0.505886  0.355421  0.25551    0.529211  0.558492
 0.41672   0.116288  0.0878837  0.649566  0.297722

In [30]:
loss(x,y)

2.561531290621767

In [31]:
W1 = rand(3,5)
b1 = rand(3)
layer1(x) = W1*x .+ b1

W2 = rand(2,3)
b2 = rand(2)
layer2(x) = W2*x .+ b2

layer2 (generic function with 1 method)

In [33]:
model(x) = layer2(σ.(layer1(x)))

model (generic function with 1 method)

In [34]:
model(rand(5))

2-element Vector{Float64}:
 2.708607068601554
 2.544804321090009

In [35]:
function linear(in,out)
    W = randn(out,in)
    b = randn(out)
    x -> W*x .+ b
end

linear (generic function with 1 method)

In [36]:
linear1 = linear(5,3)
linear2 = linear(3,2)

#5 (generic function with 1 method)

In [37]:
model(x) = linear2(σ.(linear1(x)))

model (generic function with 1 method)

In [38]:
model(rand(5))

2-element Vector{Float64}:
 -1.232166461802826
  1.0905079003829419

In [39]:
struct Affine
  W
  b
end

Affine(in::Integer, out::Integer) =
  Affine(randn(out, in), randn(out))

# Overload call, so the object can be used as a function
(m::Affine)(x) = m.W * x .+ m.b

a = Affine(10, 5)

a(rand(10)) # => 5-element vector

5-element Vector{Float64}:
 -2.0569200043481457
  0.5291964258570815
  0.022364722343165444
 -2.1226540250795836
 -1.6916634276472302

# Stacking It Up
        
It's pretty common to write models that look something like:

In [43]:
layers = [Dense(10, 5, σ), Dense(5, 2), softmax]

model(x) = foldl((x, m) -> m(x), layers, init = x)

model(rand(10)) # => 2-element vector

2-element Vector{Float64}:
 0.30598508147716963
 0.6940149185228304

In [44]:
model2 = Chain(
  Dense(10, 5, σ),
  Dense(5, 2),
  softmax)

model2(rand(10)) # => 2-element vector

2-element Vector{Float64}:
 0.6417426727013729
 0.35825732729862725

In [45]:
m = Dense(5, 2) ∘ Dense(10, 5, σ)

m(rand(10))

2-element Vector{Float64}:
  0.5298847658491741
 -0.7395216463803215

In [46]:
m = Chain(x -> x^2, x -> x+1)

m(5) # => 26

26

In [47]:
Flux.@functor Affine